# Detecção de Anomalias com Isolation Forest
## Inventory Anomaly Detector

Este notebook realiza a detecção de anomalias em consumo e estoque usando Isolation Forest.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
from datetime import datetime

# Adicionar src ao path
sys.path.insert(0, str(Path.cwd()))

from src.data_loader import load_inventory_data, load_raw_consumo
from src.data_cleaning import clean_consumo, save_processed
from src.data_aggregator import aggregate_daily_by_item
from src.anomalies import (
    train_isolation_forest,
    detect_anomalies,
    detect_anomalies_consumo_estoque,
    save_anomaly_model,
    load_anomaly_model
)
from src.alerts import (
    format_anomaly_alert,
    format_anomaly_email_html,
    send_anomaly_alerts,
    send_anomaly_alert_by_product
)

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Bibliotecas importadas com sucesso!")


## 1. Carregar e Preparar Dados


In [ ]:
# Carregar dados de estoque e consumo
df = load_inventory_data()
print(f"Shape do dataset: {df.shape}")
print(f"\nColunas: {df.columns.tolist()}")
print(f"\nPrimeiras linhas:")
df.head()


In [ ]:
# Verificar informações básicas
print("Informações do dataset:")
print(df.info())
print("\n" + "="*60)
print("Estatísticas descritivas:")
df.describe()


## 2. Agregação Diária por Item


In [ ]:
# Agregar dados diários por produto
df_aggregated = aggregate_daily_by_item(df)

print(f"\nShape após agregação: {df_aggregated.shape}")
print(f"\nColunas criadas: {df_aggregated.columns.tolist()}")
df_aggregated.head(10)


In [ ]:
# Verificar produtos únicos e período
print(f"Produtos únicos: {df_aggregated['produto_id'].nunique()}")
print(f"Período: {df_aggregated['data'].min()} a {df_aggregated['data'].max()}")
print(f"Total de dias: {(df_aggregated['data'].max() - df_aggregated['data'].min()).days + 1}")


## 3. Treinar Isolation Forest


In [ ]:
# Definir features para detecção de anomalias
# Usaremos consumo e estoque médios
feature_columns = ['consumo_mean', 'estoque_mean']

# Verificar se as colunas existem
missing_cols = [col for col in feature_columns if col not in df_aggregated.columns]
if missing_cols:
    print(f"Aviso: Colunas não encontradas: {missing_cols}")
    print(f"Colunas disponíveis: {df_aggregated.columns.tolist()}")
else:
    print(f"Features selecionadas: {feature_columns}")
    print(f"\nEstatísticas das features:")
    print(df_aggregated[feature_columns].describe())


In [ ]:
# Treinar modelo Isolation Forest
# contamination: proporção esperada de anomalias (10% = 0.1)
anomaly_model = train_isolation_forest(
    df_aggregated,
    feature_columns=feature_columns,
    contamination=0.1,  # Esperamos ~10% de anomalias
    random_state=42,
    n_estimators=100
)


## 4. Detectar Anomalias


In [ ]:
# Detectar anomalias usando o modelo treinado
df_with_anomalies = detect_anomalies(
    anomaly_model,
    df_aggregated,
    feature_columns=feature_columns
)

print(f"\nPrimeiras linhas com anomalias detectadas:")
df_with_anomalies.head(10)


In [ ]:
# Filtrar apenas as anomalias
anomalies = df_with_anomalies[df_with_anomalies['is_anomaly'] == True].copy()

print(f"Total de anomalias detectadas: {len(anomalies)}")
print(f"Percentual de anomalias: {len(anomalies)/len(df_with_anomalies)*100:.2f}%")

if len(anomalies) > 0:
    print(f"\nAnomalias detectadas:")
    print(anomalies[['produto_id', 'data', 'consumo_mean', 'estoque_mean', 'anomaly_score']].head(10))


## 5. Análise Estatística das Anomalias


In [ ]:
# Estatísticas das anomalias
if len(anomalies) > 0:
    print("Estatísticas das anomalias:")
    print("="*60)
    print(f"Score médio: {anomalies['anomaly_score'].mean():.4f}")
    print(f"Score mínimo: {anomalies['anomaly_score'].min():.4f}")
    print(f"Score máximo: {anomalies['anomaly_score'].max():.4f}")
    print(f"\nAnomalias por produto:")
    print(anomalies['produto_id'].value_counts().head(10))
    
    print(f"\nAnomalias por data:")
    anomalies_by_date = anomalies.groupby('data').size().sort_values(ascending=False)
    print(anomalies_by_date.head(10))


In [ ]:
# Comparar estatísticas: anomalias vs. normal
print("Comparação: Anomalias vs. Dados Normais")
print("="*60)

normal_data = df_with_anomalies[df_with_anomalies['is_anomaly'] == False]

comparison = pd.DataFrame({
    'Normal': [
        normal_data['consumo_mean'].mean(),
        normal_data['consumo_mean'].std(),
        normal_data['estoque_mean'].mean(),
        normal_data['estoque_mean'].std()
    ],
    'Anomalias': [
        anomalies['consumo_mean'].mean(),
        anomalies['consumo_mean'].std(),
        anomalies['estoque_mean'].mean(),
        anomalies['estoque_mean'].std()
    ]
}, index=['Consumo Médio', 'Consumo Std', 'Estoque Médio', 'Estoque Std'])

print(comparison)


## 6. Visualização das Anomalias


In [ ]:
# Gráfico 1: Distribuição de scores de anomalia
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Score de anomalia
axes[0, 0].hist(df_with_anomalies['anomaly_score'], bins=50, alpha=0.7, edgecolor='black')
axes[0, 0].axvline(df_with_anomalies['anomaly_score'].mean(), color='red', linestyle='--', label='Média')
axes[0, 0].set_title('Distribuição de Scores de Anomalia')
axes[0, 0].set_xlabel('Anomaly Score')
axes[0, 0].set_ylabel('Frequência')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Consumo vs Estoque (todos os dados)
scatter = axes[0, 1].scatter(
    df_with_anomalies['consumo_mean'],
    df_with_anomalies['estoque_mean'],
    c=df_with_anomalies['anomaly_score'],
    cmap='RdYlGn_r',
    alpha=0.6,
    s=50
)
axes[0, 1].scatter(
    anomalies['consumo_mean'],
    anomalies['estoque_mean'],
    color='red',
    marker='x',
    s=100,
    label='Anomalias',
    linewidths=2
)
axes[0, 1].set_title('Consumo vs Estoque (colorido por score)')
axes[0, 1].set_xlabel('Consumo Médio')
axes[0, 1].set_ylabel('Estoque Médio')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[0, 1], label='Anomaly Score')

# Série temporal de consumo com anomalias destacadas
if len(anomalies) > 0:
    # Pegar um produto como exemplo
    produto_exemplo = anomalies['produto_id'].iloc[0]
    df_produto = df_with_anomalies[df_with_anomalies['produto_id'] == produto_exemplo].sort_values('data')
    anomalias_produto = anomalies[anomalies['produto_id'] == produto_exemplo].sort_values('data')
    
    axes[1, 0].plot(df_produto['data'], df_produto['consumo_mean'], label='Consumo', alpha=0.7)
    axes[1, 0].scatter(anomalias_produto['data'], anomalias_produto['consumo_mean'], 
                      color='red', marker='x', s=100, label='Anomalias', linewidths=2)
    axes[1, 0].set_title(f'Série Temporal - Consumo (Produto: {produto_exemplo})')
    axes[1, 0].set_xlabel('Data')
    axes[1, 0].set_ylabel('Consumo Médio')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].tick_params(axis='x', rotation=45)

# Série temporal de estoque com anomalias destacadas
axes[1, 1].plot(df_produto['data'], df_produto['estoque_mean'], label='Estoque', alpha=0.7, color='green')
axes[1, 1].scatter(anomalias_produto['data'], anomalias_produto['estoque_mean'], 
                   color='red', marker='x', s=100, label='Anomalias', linewidths=2)
axes[1, 1].set_title(f'Série Temporal - Estoque (Produto: {produto_exemplo})')
axes[1, 1].set_xlabel('Data')
axes[1, 1].set_ylabel('Estoque Médio')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Gráfico 2: Anomalias por produto e por data
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Anomalias por produto
if len(anomalies) > 0:
    anomalies_by_product = anomalies['produto_id'].value_counts().head(10)
    axes[0].barh(range(len(anomalies_by_product)), anomalies_by_product.values)
    axes[0].set_yticks(range(len(anomalies_by_product)))
    axes[0].set_yticklabels(anomalies_by_product.index)
    axes[0].set_title('Top 10 Produtos com Mais Anomalias')
    axes[0].set_xlabel('Número de Anomalias')
    axes[0].grid(True, alpha=0.3, axis='x')
    
    # Anomalias por data
    anomalies_by_date = anomalies.groupby('data').size().sort_values(ascending=False).head(10)
    axes[1].bar(range(len(anomalies_by_date)), anomalies_by_date.values)
    axes[1].set_xticks(range(len(anomalies_by_date)))
    axes[1].set_xticklabels([d.strftime('%Y-%m-%d') for d in anomalies_by_date.index], rotation=45)
    axes[1].set_title('Top 10 Datas com Mais Anomalias')
    axes[1].set_ylabel('Número de Anomalias')
    axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 7. Formatação de Alertas


In [ ]:
# Formatar mensagem de alerta (texto)
if len(anomalies) > 0:
    alert_message = format_anomaly_alert(anomalies.head(10), max_anomalies=10)
    print("Mensagem de alerta formatada:")
    print("="*60)
    print(alert_message)


In [ ]:
# Formatar mensagem HTML para email
if len(anomalies) > 0:
    html_message = format_anomaly_email_html(anomalies.head(20))
    print("HTML gerado (primeiras 500 caracteres):")
    print("="*60)
    print(html_message[:500])
    print("...")


## 8. Envio de Alertas (Exemplo)


In [ ]:
# Exemplo de envio de alertas
# NOTA: Configure os webhooks/email em src/config.py antes de usar

if len(anomalies) > 0:
    # Filtrar apenas anomalias com score alto (>= 0.7)
    high_score_anomalies = anomalies[anomalies['anomaly_score'] >= 0.7]
    
    if len(high_score_anomalies) > 0:
        print(f"Anomalias com score alto (>= 0.7): {len(high_score_anomalies)}")
        print("\nPara enviar alertas, descomente as linhas abaixo e configure:")
        print("  - Discord webhook em src/config.py")
        print("  - Teams webhook em src/config.py")
        print("  - Email SMTP em src/config.py")
        
        # Exemplo (descomentar para usar):
        # results = send_anomaly_alerts(
        #     high_score_anomalies,
        #     produto_id="TODOS",
        #     min_score=0.7,
        #     send_discord=False,  # Configure webhook primeiro
        #     send_teams=False,    # Configure webhook primeiro
        #     send_email=False     # Configure SMTP primeiro
        # )
        # print(f"\nResultados do envio: {results}")
    else:
        print("Nenhuma anomalia com score >= 0.7 encontrada.")
else:
    print("Nenhuma anomalia detectada.")


In [ ]:
# Enviar alertas por produto (exemplo)
# NOTA: Configure os webhooks/email em src/config.py antes de usar

if len(anomalies) > 0:
    print("Para enviar alertas por produto, descomente as linhas abaixo:")
    
    # Exemplo (descomentar para usar):
    # results_by_product = send_anomaly_alert_by_product(
    #     anomalies,
    #     produto_column="produto_id",
    #     min_score=0.7,
    #     send_discord=False,
    #     send_teams=False,
    #     send_email=False
    # )
    # print(f"\nAlertas enviados para {len(results_by_product)} produtos")


## 9. Salvar Resultados


In [ ]:
# Salvar modelo de anomalias
model_path = Path("outputs/models/isolation_forest_model.pkl.gz")
save_anomaly_model(anomaly_model, model_path, compress=True)


In [ ]:
# Salvar todos os dados com flag de anomalia
output_path_all = Path("outputs/anomalies_detected.csv")
save_processed(df_with_anomalies, output_path_all, format="csv")
print(f"Dados completos salvos em: {output_path_all}")

# Salvar em Parquet (mais eficiente)
output_path_parquet = Path("outputs/anomalies_detected.parquet")
save_processed(df_with_anomalies, output_path_parquet, format="parquet", compress=True)
print(f"Dados completos salvos em Parquet: {output_path_parquet}")


In [ ]:
# Salvar apenas as anomalias
if len(anomalies) > 0:
    output_path_anomalies = Path("outputs/anomalies_only.csv")
    save_processed(anomalies, output_path_anomalies, format="csv")
    print(f"Apenas anomalias salvas em: {output_path_anomalies}")
    
    # Salvar em Parquet
    output_path_anomalies_parquet = Path("outputs/anomalies_only.parquet")
    save_processed(anomalies, output_path_anomalies_parquet, format="parquet", compress=True)
    print(f"Apenas anomalias salvas em Parquet: {output_path_anomalies_parquet}")
else:
    print("Nenhuma anomalia para salvar.")


## 10. Carregar Modelo Salvo (Exemplo)


In [ ]:
# Exemplo de como carregar um modelo salvo
# loaded_model = load_anomaly_model(Path("outputs/models/isolation_forest_model.pkl.gz"))
# print("Modelo carregado com sucesso!")

# Usar modelo carregado para detectar anomalias em novos dados
# df_new = ...  # Seus novos dados
# df_new_anomalies = detect_anomalies(loaded_model, df_new, feature_columns=feature_columns)
